# Wikipedia Question Answering

In [12]:
import os
import json

import wikipedia

from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from openai import OpenAI

EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-4"

In [17]:
client = OpenAI(
    api_key="mabeleda",
    base_url="http://openai-api-proxy.discovery:8888/v1",
)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [8]:
# query = "How many goals did Lionel Messi score in the 2008/09 season?"

In [29]:
query = "How did the Roman Empire fall?"

In [57]:
query = "Can you summarize the investing thesis for Tesla?"

In [58]:
message = f"""
Given the following query, generate a set of 3 searches for Wikipedia which are most likely to find this information.
Return your search terms as a json blob with the attribute 'search_terms' which is a list of the search terms to use.
Ensure your query terms are different enough to search a wide variety of pages.

Query: {query}
"""
print(message)
messages = [
    {
        "role": "system",
        "content": "You generate search terms to query from Wikipedia.",
    },
    {"role": "user", "content": message},
]
completion = client.chat.completions.create(
    model=GPT_MODEL,
    messages=messages,
    temperature=0,
)

queries = json.loads(completion.choices[0].message.content)
queries["search_terms"]


Given the following query, generate a set of 3 searches for Wikipedia which are most likely to find this information.
Return your search terms as a json blob with the attribute 'search_terms' which is a list of the search terms to use.
Ensure your query terms are different enough to search a wide variety of pages.

Query: Can you summarize the investing thesis for Tesla?



['Tesla company investment analysis',
 'Tesla stock market performance',
 'Elon Musk business strategy']

In [59]:
results = []
for search in queries['search_terms']:
    results.append(wikipedia.run(search))
articles = '\n'.join(results)

In [60]:
message = f"""
Articles:

{articles}

Query: {query}
"""
print(message)
messages = [
    {
        "role": "system",
        "content": "You generate answers to a query based on article content from wikipedia. Be concise. Add detail only where necessary, I'll let you be the judge of that. Never tell me that as an AI you can't provide advice. I know you are an AI so I don't make decisions directly from your advice",
    },
    {"role": "user", "content": message},
]
completion = client.chat.completions.create(
    model=GPT_MODEL,
    messages=messages,
    temperature=0,
)
print(completion.choices[0].message.content)


Articles:

Page: Tesla, Inc.
Summary: Tesla, Inc. ( TESS-lə or  TEZ-lə) is an American multinational automotive and clean energy company headquartered in Austin, Texas, which designs and manufactures electric vehicles (cars and trucks), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services. Its subsidiary Tesla Energy develops and is a major installer of photovoltaic systems in the United States and is one of the largest global suppliers of battery energy storage systems with 6.5 gigawatt-hours (GWh) installed in 2022.
Tesla is one of the world's most valuable companies and, as of 2023, is the world's most valuable automaker. In 2022, the company led the battery electric vehicle market, with 18% share.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. The company's name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, via a $6.5 mill